In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib firebase-admin

   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   - -------------------------------------- 0.4/13.7 MB 12.2 MB/s eta 0:00:02
   -- ------------------------------------- 0.9/13.7 MB 11.9 MB/s eta 0:00:02
   ---- ----------------------------------- 1.5/13.7 MB 11.8 MB/s eta 0:00:02
   ----- ---------------------------------- 2.0/13.7 MB 11.7 MB/s eta 0:00:02
   ------- -------------------------------- 2.6/13.7 MB 11.7 MB/s eta 0:00:01
   --------- ------------------------------ 3.1/13.7 MB 11.6 MB/s eta 0:00:01
   ---------- ----------------------------- 3.6/13.7 MB 11.7 MB/s eta 0:00:01
   ------------ --------------------------- 4.2/13.7 MB 11.6 MB/s eta 0:00:01
   ------------- -------------------------- 4.7/13.7 MB 12.0 MB/s eta 0:00:01
   --------------- ------------------------ 5.2/13.7 MB 12.0 MB/s eta 0:00:01
   ---------------- ----------------------- 5.8/13.7 MB 11.9 MB/s eta 0:00:01
   ------------------ --------------------- 6.3/13.7 MB 11.9 MB/s eta 0:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 6.31.1 which is incompatible.


In [3]:
!pip install protobuf==3.20.*
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib firebase-admin

   ---------------------------------------- 0.0/162.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/162.1 kB ? eta -:--:--
   ------- ------------------------------- 30.7/162.1 kB 660.6 kB/s eta 0:00:01
   ------------------------------ --------- 122.9/162.1 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 162.1/162.1 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.74.0 requires protobuf<7.0.0,>=6.31.1, but you have protobuf 3.20.3 which is incompatible.


  Using cached protobuf-6.31.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)
Using cached protobuf-6.31.1-cp310-abi3-win_amd64.whl (435 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 6.31.1 which is incompatible.


Go to this URL: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=970036876125-ldoi22mcoc5mbnlf80cda2d3hjocosto.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=C2l5WZE8yTWhSR1kSRJ5JrjzmAE3ml&prompt=consent&access_type=offline


Enter authorization code:  ㅇ


InvalidGrantError: (invalid_grant) Malformed auth code.

In [3]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def get_gmail_service():
    flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
    creds = flow.run_local_server(port=0)  # localhost에서 인증 완료
    service = build('gmail', 'v1', credentials=creds)
    return service

service = get_gmail_service()

results = service.users().messages().list(userId='me', maxResults=5).execute()
messages = results.get('messages', [])
for msg in messages:
    print(f'Message ID: {msg["id"]}')


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=970036876125-ldoi22mcoc5mbnlf80cda2d3hjocosto.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53908%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=VKz8Hu4AHHid5R5g1HYSlKeuaWEwAU&access_type=offline
Message ID: 19843ad921cad879
Message ID: 19840393a05ef586
Message ID: 1983dcbb3e6ca5e6
Message ID: 1983973d2e5a3fc2
Message ID: 198374951b095023


In [4]:
import base64
from googleapiclient.discovery import build

def get_message_body(service, user_id, msg_id):
    message = service.users().messages().get(userId=user_id, id=msg_id, format='full').execute()

    payload = message.get('payload', {})
    parts = payload.get('parts', [])
    
    # 본문이 바로 payload에 있을 수도 있으니 체크
    if 'body' in payload and 'data' in payload['body']:
        data = payload['body']['data']
    else:
        # multipart일 경우 첫 번째 파트의 본문을 가져옴
        if len(parts) > 0:
            data = parts[0]['body']['data']
        else:
            data = None
    
    if data:
        # Base64 URL safe 디코딩
        decoded_bytes = base64.urlsafe_b64decode(data.encode('UTF-8'))
        return decoded_bytes.decode('UTF-8')
    else:
        return None

# 사용 예시
user_id = 'me'
for msg in messages:  # messages는 이전에 가져온 메일 리스트
    msg_id = msg['id']
    body = get_message_body(service, user_id, msg_id)
    print(f'Message ID: {msg_id}\nBody:\n{body}\n{"-"*50}')


Message ID: 19843ad921cad879
Body:
[image: Google]
Android에서 새로 로그인함


th051464@gmail.com
Android 기기에서 내 Google 계정에 새로 로그인했습니다. 직접 로그인한 것이  
맞다면 아무런 조치를 취하지 않아도 됩니다.
본인이 아니라면 안내에 따라 계정을 보호하세요.
활동 확인
<https://accounts.google.com/AccountChooser?Email=th051464@gmail.com&continue=https://myaccount.google.com/alert/nt/1753482103000?rfn%3D325%26rfnc%3D1%26eid%3D4504461712158761340%26et%3D0>
다음 페이지에서 보안 활동도 확인할 수 있습니다.
https://myaccount.google.com/notifications
이 이메일은 Google 계정 및 서비스의 중요한 변경사항을 알려드리기 위해  
발송되었습니다.
© 2025 Google LLC, 1600 Amphitheatre Parkway, Mountain View, CA 94043, USA

--------------------------------------------------
Message ID: 19840393a05ef586
Body:
Read the latest tutorials and news curated for you.
Join NVIDIA at SIGGRAPH 2025. Learn more ❯ <https://go.nvidianews.com/MTU2LU9GTi03NDIAAAGb3yvu9LSsYyOUzozEFyjgnk0s5PSXXPxVRCOiuMwbSdn5UW1v-YItz2AAOkKZWQmjK9UqFQ0=>
This newsletter was curated based on your topic preferences. Click here to update <https://go.nvidianews.com/

In [6]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import base64
from bs4 import BeautifulSoup

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def get_gmail_service():
    flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
    creds = flow.run_local_server(port=0)
    service = build('gmail', 'v1', credentials=creds)
    return service

def get_message_body(service, user_id, msg_id):
    message = service.users().messages().get(userId=user_id, id=msg_id, format='full').execute()
    payload = message.get('payload', {})
    parts = payload.get('parts', [])

    # 본문 데이터 추출 함수
    def get_part_body(parts):
        for part in parts:
            if part.get('mimeType') == 'text/plain':
                data = part['body'].get('data')
                if data:
                    return base64.urlsafe_b64decode(data).decode('utf-8')
            elif part.get('mimeType') == 'text/html':
                data = part['body'].get('data')
                if data:
                    html = base64.urlsafe_b64decode(data).decode('utf-8')
                    # HTML 태그 제거
                    return BeautifulSoup(html, 'html.parser').get_text()
            elif part.get('parts'):
                # 재귀 탐색
                return get_part_body(part.get('parts'))
        return None

    # 만약 parts 없으면 payload body 확인
    if parts:
        body = get_part_body(parts)
    else:
        body_data = payload.get('body', {}).get('data')
        if body_data:
            body = base64.urlsafe_b64decode(body_data).decode('utf-8')
        else:
            body = None
    return body

service = get_gmail_service()

query = '"Youtube Premium"'
results = service.users().messages().list(userId='me', q=query, maxResults=10).execute()
messages = results.get('messages', [])

for msg in messages:
    msg_id = msg['id']
    body = get_message_body(service, 'me', msg_id)
    
    # 메일 제목 가져오기
    message = service.users().messages().get(userId='me', id=msg_id, format='metadata', metadataHeaders=['Subject']).execute()
    headers = message.get('payload', {}).get('headers', [])
    subject = next((h['value'] for h in headers if h['name'] == 'Subject'), '(No Subject)')

    print(f"Subject: {subject}")
    print(f"Body snippet:\n{body[:500]}")  # 최대 500자만 출력
    print('-'*50)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=970036876125-ldoi22mcoc5mbnlf80cda2d3hjocosto.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A54133%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=0MudRUtVhEpIIxodcejCZ99Z4rXEUv&access_type=offline
Subject: 축하합니다. YouTube Premium 회원이 되셨습니다

Body snippet:

   
<https://notifications.google.com/g/p/ANiao5ppvmtL6kI9tsFCiHGdzvtQndieNa2wYEn4GmkfvdKbfZCdjjzPfwQW9uFZMdlBMpAndq0Pp3yxKUWtWLt1ySTVMb_DS6eg8UfffW-1yMHuUUBMLXhHKf-FFtGj3RGdEIDZ_0-BHoM>

-----

YOUTUBE PREMIUM에 오신 것을 환영합니다YouTube와 YouTube Music을 광고의 방해  
없이 즐기세요
지금 보기  
<https://notifications.google.com/g/p/ANiao5pafqOciQVnBnp8UjXDsr4PPni5GomekyJaWaWPmM8uWr0w3obK9GktPIv_IMnuXt5vPjxGTeg55t7s1ogwSHEVTbcmVJcem7DKv_KaXIvMI7ZlsuYjkELmtNvy_yVkTeMho3mncw>
회원 혜택을 모두 확인해 보세요


재생 중인 YouTub
--------------------------------------------------
Subject: Apple에서 발행한 영수증입니다.
Body snippe

In [1]:
import base64
from bs4 import BeautifulSoup
from datetime import datetime
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import firebase_admin
from firebase_admin import credentials, firestore
from email.utils import parsedate_to_datetime

# 1. Gmail API 인증
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
creds = flow.run_local_server(port=0)
gmail_service = build('gmail', 'v1', credentials=creds)

# 2. Firestore 연결 (중복 초기화 방지)
if not firebase_admin._apps:
    cred = credentials.Certificate("firebase-key.json")
    firebase_admin.initialize_app(cred)
db = firestore.client()

# 3. 본문 텍스트 추출 함수
def get_message_body(service, user_id, msg_id):
    message = service.users().messages().get(userId=user_id, id=msg_id, format='full').execute()
    payload = message.get('payload', {})
    parts = payload.get('parts', [])

    if not parts and 'body' in payload:
        data = payload['body'].get('data')
        if data:
            decoded_bytes = base64.urlsafe_b64decode(data.encode('UTF-8'))
            return decoded_bytes.decode('UTF-8')

    for part in parts:
        mime_type = part.get('mimeType')
        data = part.get('body', {}).get('data')
        if data:
            decoded_bytes = base64.urlsafe_b64decode(data.encode('UTF-8'))
            if mime_type == 'text/plain':
                return decoded_bytes.decode('UTF-8')
            elif mime_type == 'text/html':
                soup = BeautifulSoup(decoded_bytes, 'html.parser')
                return soup.get_text()
    return ""

# 4. 본문에서 필요한 정보 파싱 함수 (제목 제외)
def parse_subscription_info(body):
    info = {
        "서비스명": None,
        "구입 날짜": None,
        "갱신 가격": None
    }

    lines = [line.strip() for line in body.split('\n') if line.strip()]
    
    for i, line in enumerate(lines):
        if line.startswith("앱"):
            value = line[2:].strip() or (lines[i+1].strip() if i+1 < len(lines) else None)
            info["서비스명"] = value

        elif line.startswith("구입 날짜"):
            value = line[len("구입 날짜"):].strip() or (lines[i+1].strip() if i+1 < len(lines) else None)
            info["구입 날짜"] = value

        elif line.startswith("갱신 가격"):
            value = line[len("갱신 가격"):].strip() or (lines[i+1].strip() if i+1 < len(lines) else None)
            info["갱신 가격"] = value

    return info

# 5. "구독 확인"이 포함된 메일 검색
query = 'subject:"구독 확인"'
results = gmail_service.users().messages().list(userId='me', q=query, maxResults=20).execute()
messages = results.get('messages', [])

# 6. 메일 처리
for msg in messages:
    msg_id = msg['id']
    message_detail = gmail_service.users().messages().get(userId='me', id=msg_id, format='full').execute()

    # 제목과 날짜 추출
    headers = message_detail['payload'].get('headers', [])
    subject = next((h['value'] for h in headers if h['name'].lower() == 'subject'), '(제목 없음)')
    date_raw = next((h['value'] for h in headers if h['name'].lower() == 'date'), '')
    
    try:
        date_obj = parsedate_to_datetime(date_raw)
        date_str = date_obj.strftime('%Y-%m-%d')
    except Exception as e:
        print(f"날짜 파싱 실패: {e}")
        date_str = date_raw

    body = get_message_body(gmail_service, 'me', msg_id)

    # 본문 필수 키워드 체크
    required_keywords = ["앱", "구입 날짜", "갱신 가격"]
    if not all(keyword in body for keyword in required_keywords):
        print(f"메시지 {msg_id}는 필수 키워드 부족으로 건너뜁니다.")
        continue

    parsed_info = parse_subscription_info(body)

    # 출력
    print(f"\n📩 제목: {subject}")
    print(f"🗓 날짜: {date_str}")
    print(f"🛎 서비스명: {parsed_info.get('서비스명', '알 수 없음')}")
    print(f"구입 날짜: {parsed_info.get('구입 날짜', '알 수 없음')}")
    print(f"갱신 가격: {parsed_info.get('갱신 가격', '알 수 없음')}")

    # Firestore 저장 (제목, 날짜, 서비스명, 구입 날짜, 갱신 가격만)
    doc = {
        'subject': subject,
        'date': date_str,
        'service_name': parsed_info.get('서비스명', '알 수 없음'),
        'purchase_date': parsed_info.get('구입 날짜', '알 수 없음'),
        'renewal_price': parsed_info.get('갱신 가격', '알 수 없음'),
    }
    db.collection('subscriptions').document(msg_id).set(doc)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=970036876125-ldoi22mcoc5mbnlf80cda2d3hjocosto.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A62377%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=aDOdp0WqNhnFJcitNpHbpAAmHOTfHR&access_type=offline

📩 제목: 구독 확인
🗓 날짜: 2025-05-04
🛎 서비스명: YouTube
구입 날짜: 2025년 5월 4일
갱신 가격: ₩19,500/1개월

📩 제목: 구독 확인
🗓 날짜: 2025-04-18
🛎 서비스명: 메모위젯 (할일 & 포토위젯)
구입 날짜: 2025년 4월 18일
갱신 가격: ₩14,000/1년
메시지 18e9d1efbe9eb5b5는 필수 키워드 부족으로 건너뜁니다.
메시지 18e1d9e7fc171749는 필수 키워드 부족으로 건너뜁니다.
메시지 18c2b1ae25e6a89f는 필수 키워드 부족으로 건너뜁니다.
